# Homework 6

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.neighbors import NearestNeighbors

In [3]:
df = pd.read_csv('homework_6.1.csv', index_col=0)

In [4]:
df.head()

,Z,X,Y
0,0.548814,0,-0.823220
1,0.715189,1,0.842405
2,0.602763,1,0.898618
3,0.544883,0,-0.817325
4,0.423655,0,-0.635482


In [6]:

# Split into treated and untreated
treated = df[df['X'] == 1].reset_index(drop=True)
control = df[df['X'] == 0].reset_index(drop=True)

# --- Helper function: nearest neighbor matching ---
def match_groups(source, target, source_label="treated", target_label="control"):
    """
    For each row in source, find the nearest neighbor in target based on Z.
    Returns a DataFrame with source Y, matched target Y, and the difference.
    """
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(target[['Z']])
    distances, indices = nn.kneighbors(source[['Z']])
    
    matched = target.iloc[indices.flatten()].reset_index(drop=True)
    diffs = source['Y'].values - matched['Y'].values
    
    return pd.DataFrame({
        f'{source_label}_Y': source['Y'].values,
        f'{target_label}_Y': matched['Y'].values,
        'diff': diffs
    })

# --- 1. Average Treatment Effect (ATE) ---
# Each item (treated and untreated) gets a counterfactual
treated_matches = match_groups(treated, control, "treated", "control")
control_matches = match_groups(control, treated, "control", "treated")

ate = pd.concat([treated_matches['diff'], -control_matches['diff']]).mean()
print("ATE (average treatment effect):", ate)

# --- 2. Average Treatment Effect on the Treated (ATT) ---
att = treated_matches['diff'].mean()
print("ATT (average treatment effect on the treated):", att)

# --- 3. Average Treatment Effect on the Untreated (ATU) ---
atu = -control_matches['diff'].mean()
print("ATU (average treatment effect on the untreated):", atu)

# --- 4. Optimal Treatment Effect ---
# For each untreated, find its matched treated difference, then take the maximum
optimal_te = (-control_matches['diff']).max()
print("Optimal Treatment Effect:", optimal_te)

ATE (average treatment effect): 1.6952701427487222
ATT (average treatment effect on the treated): 1.846408507179443
ATU (average treatment effect on the untreated): 1.5494765534845105
Optimal Treatment Effect: 2.1724698851117035
